In [2]:
import numpy as np
import xarray as xr
import pandas as pd 

In [7]:
ivarn = 0

# set parameters for each run
nmem = 1  # memory time steps
yrsta = 1979  # starting year in training
varnum = 1  # number of input maps
yrlen = 36  # how many years in the training dataset. 94 is actually 114. 
flg = 'trop'  # 'trop' means [-20, 20], '60deg' means [-60, 60]
varn = ["olr"]  # "prep", 

fnROMI = '/global/homes/l/linyaoly/ERA5/reanalysis/cal_ROMI/ROMI_ERA5_daily_'+str(yrsta)+'to2021.nc'  # file path for the obs ROMI
dirn = '/global/homes/l/linyaoly/ERA5/script/Unet4MJO/'+str(varnum)+'maps_MCDO_'+str(yrlen)+'yrtraining_ROMIERA5_filtered_'+flg+'/ROMI_dp0102_1dmem_'+varn[ivarn]+'/'  # directory path for the output

lead = [1, 5, 10, 15, 20, 25, 28, 30, 33]   # select the prediction lead
# lead = [1, 5, 10, 13, 15, 25, 30]   # select the prediction lead
# get initial phase

dsp = []
for yn in np.arange(2015,2020):
    fnp=dirn + 'predicted_dis_MCDO_UNET_'+str(varn[ivarn])+flg+'_ROMIERA5_'+str(yrlen)+'yr_lead' + str(1) + '_dailyinput_mem'+str(nmem)+'d' + str(yn) + '.nc'
    fnt=dirn + 'truth_MCDO_UNET_'+str(varn[ivarn])+flg+'_ROMIERA5_'+str(yrlen)+'yr_lead' + str(1) + '_dailyinput_mem'+str(nmem)+'d' + str(yn) + '.csv'

    df = xr.open_dataset(fnp)
    dsp.append(df['ROMI'])
    del df 

dsp0 = xr.concat(dsp, dim='timestp')

iniROMI = xr.open_dataset(fnROMI)

if nmem == 1:
    a = iniROMI.sel(time=slice('2015-01-01','2016-12-30'))
    b = iniROMI.sel(time=slice('2017-01-01','2019-12-31'))
elif nmem == 30:
    a = iniROMI.sel(time=slice('2015-01-30','2017-01-28'))
    b = iniROMI.sel(time=slice('2017-01-30','2020-01-29'))

iniROMI = xr.concat((a,b), dim='time')
del a 
del b 
ROMI1 = iniROMI['ROMI'][:,0]
ROMI2 = iniROMI['ROMI'][:,1]
IniAmp = np.sqrt(ROMI1 * ROMI1 + ROMI2 * ROMI2)
# dsp0['timestp'] = IniAmp
dsp0 

<xarray.DataArray 'ROMI' (timestp: 1825, N: 100, mode: 2)>
array([[[-1.14588809,  0.24512114],
        [-1.50624025,  0.27003884],
        [-1.62744772,  0.3727673 ],
        ...,
        [-1.30064142,  0.42934403],
        [-1.48208368,  0.43993008],
        [-1.40408432,  0.22666778]],

       [[-1.80500448,  0.3123678 ],
        [-1.84430385,  0.12767686],
        [-1.72848189,  0.09847084],
        ...,
        [-1.54798234,  0.07813962],
        [-1.43071747,  0.22718866],
        [-2.0180881 ,  0.38888708]],

       [[-2.05522752, -0.1000496 ],
        [-1.97971058,  0.32863903],
        [-1.87690914,  0.28274441],
        ...,
...
        ...,
        [-0.51978052,  0.0381789 ],
        [-0.51205385,  0.04982163],
        [-0.67386025,  0.10110171]],

       [[-0.54182124,  0.08427894],
        [-0.59965503,  0.25882193],
        [-0.68662935,  0.16444005],
        ...,
        [-0.61180872, -0.07573647],
        [-0.62407523,  0.20824161],
        [-0.5193885 ,  0.09394032]],

       [[-0.90156591,  0.09399059],
        [-0.49726611,  0.29942513],
        [-0.51956528,  0.18074618],
        ...,
        [-0.49178427,  0.16297613],
        [-0.68106598,  0.21909004],
        [-0.68623191,  0.1680955 ]]])
Coordinates:
  * timestp  (timestp) int64 0 1 2 3 4 5 6 7 ... 357 358 359 360 361 362 363 364
  * N        (N) int64 0 1 2 3 4 5 6 7 8 9 10 ... 90 91 92 93 94 95 96 97 98 99
  * mode     (mode) int64 0 1

In [11]:
np.shape(IniAmp)

(1825,)

In [18]:
IniAmp['time']

<xarray.DataArray 'time' (time: 1825)>
array(['2015-01-01T00:00:00.000000000', '2015-01-02T00:00:00.000000000',
       '2015-01-03T00:00:00.000000000', ..., '2019-12-29T00:00:00.000000000',
       '2019-12-30T00:00:00.000000000', '2019-12-31T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2019-12-31

In [17]:
dsp0.assign_coords(timestp=IniAmp['time'].values)
dsp0

<xarray.DataArray 'ROMI' (timestp: 1825, N: 100, mode: 2)>
array([[[-1.14588809,  0.24512114],
        [-1.50624025,  0.27003884],
        [-1.62744772,  0.3727673 ],
        ...,
        [-1.30064142,  0.42934403],
        [-1.48208368,  0.43993008],
        [-1.40408432,  0.22666778]],

       [[-1.80500448,  0.3123678 ],
        [-1.84430385,  0.12767686],
        [-1.72848189,  0.09847084],
        ...,
        [-1.54798234,  0.07813962],
        [-1.43071747,  0.22718866],
        [-2.0180881 ,  0.38888708]],

       [[-2.05522752, -0.1000496 ],
        [-1.97971058,  0.32863903],
        [-1.87690914,  0.28274441],
        ...,
...
        ...,
        [-0.51978052,  0.0381789 ],
        [-0.51205385,  0.04982163],
        [-0.67386025,  0.10110171]],

       [[-0.54182124,  0.08427894],
        [-0.59965503,  0.25882193],
        [-0.68662935,  0.16444005],
        ...,
        [-0.61180872, -0.07573647],
        [-0.62407523,  0.20824161],
        [-0.5193885 ,  0.09394032]],

       [[-0.90156591,  0.09399059],
        [-0.49726611,  0.29942513],
        [-0.51956528,  0.18074618],
        ...,
        [-0.49178427,  0.16297613],
        [-0.68106598,  0.21909004],
        [-0.68623191,  0.1680955 ]]])
Coordinates:
  * timestp  (timestp) int64 0 1 2 3 4 5 6 7 ... 357 358 359 360 361 362 363 364
  * N        (N) int64 0 1 2 3 4 5 6 7 8 9 10 ... 90 91 92 93 94 95 96 97 98 99
  * mode     (mode) int64 0 1

In [20]:
ds = xr.Dataset(
    data_vars=dict(
        ROMI=(["time", "N", "mode"], dsp0.values),
        IniAmp=(["time"], IniAmp.values),
    ),
    coords=dict(
        time=IniAmp.time,
        N=dsp0.N,
        mode=dsp0.mode,
    ),
    attrs=dict(description="Weather related data."),
)

In [21]:
ds 

<xarray.Dataset>
Dimensions:  (time: 1825, N: 100, mode: 2)
Coordinates:
  * time     (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2019-12-31
  * N        (N) int64 0 1 2 3 4 5 6 7 8 9 10 ... 90 91 92 93 94 95 96 97 98 99
  * mode     (mode) int64 0 1
Data variables:
    ROMI     (time, N, mode) float64 -1.146 0.2451 -1.506 ... -0.6862 0.1681
    IniAmp   (time) float64 1.834 1.971 2.131 2.227 ... 0.3278 0.5107 0.6237
Attributes:
    description:  Weather related data.

In [25]:
ds.where(ds.IniAmp>1).dropna('time')

<xarray.Dataset>
Dimensions:  (time: 1037, N: 100, mode: 2)
Coordinates:
  * time     (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2019-12-15
  * N        (N) int64 0 1 2 3 4 5 6 7 8 9 10 ... 90 91 92 93 94 95 96 97 98 99
  * mode     (mode) int64 0 1
Data variables:
    ROMI     (time, N, mode) float64 -1.146 0.2451 -1.506 ... 0.4909 0.692
    IniAmp   (time) float64 1.834 1.971 2.131 2.227 ... 1.127 1.041 1.02 1.012
Attributes:
    description:  Weather related data.